In [1]:
"""
Various helper functions.

Note:
    The functions are in alphabetical order.
"""
import numpy as np


def batch_kron(a: np.ndarray, b: np.ndarray):
    """
    Batch Kronecker product: np.kron(a[i, :, :], b[i, :, :]) for all i.
    """
    return np.einsum('bik,bjl->bijkl', a, b).reshape(a.shape[0], a.shape[1] * b.shape[1], a.shape[2] * b.shape[2])


def cov2cov(matrices: np.ndarray):
    """
    Convert the real representations of complex covariance matrices back to
    complex representations.
    """
    if matrices.ndim == 2:
        # the case of diagonal matrices
        n_mats, n_diag = matrices.shape
        mats = np.zeros([n_mats, n_diag, n_diag])
        for i in range(n_mats):
            mats[i, :, :] = np.diag(matrices[i, :])
    else:
        mats = matrices

    n_mats, rows, columns = mats.shape
    row_half = rows // 2
    column_half = columns // 2
    covs = np.zeros((n_mats, row_half, column_half), dtype=complex)
    for c in range(n_mats):
        upper_left_block = mats[c, :row_half, :column_half]
        upper_right_block = mats[c, :row_half, column_half:]
        lower_left_block = mats[c, row_half:, :column_half]
        lower_right_block = mats[c, row_half:, column_half:]
        covs[c, :, :] = upper_left_block + lower_right_block + 1j * (lower_left_block - upper_right_block)
    return covs


def cplx2real(vec: np.ndarray, axis=0):
    """
    Concatenate real and imaginary parts of vec along axis=axis.
    """
    return np.concatenate([vec.real, vec.imag], axis=axis)


def crandn(*arg, rng=np.random.default_rng()):
    return np.sqrt(0.5) * (rng.standard_normal(arg) + 1j * rng.standard_normal(arg))


# def crandn(*args, rng=None):
#     # Create a TensorFlow random generator with the given seed
#     if rng is None:
#         tf.random.set_seed(1235428719812346)
#         rng = tf.random.Generator.from_seed()
    
#     real_part = tf.sqrt(0.5) * rng.normal(shape=args)
#     imag_part = tf.sqrt(0.5) * rng.normal(shape=args)

#     print('dtype of tf.complex(real_part, imag_part): ', tf.complex(real_part, imag_part).dtype)
    
#     return tf.complex(real_part, imag_part)


def kron_approx_sep_ls(
    mats_A: np.ndarray,
    init_C: np.ndarray,
    rows_B: int,
    cols_B: int,
    iterations: int = 10
):
    """
    Approximate a matrix in terms of a Kronecker product of two matrices. The
    array init_C is an initialization for the matrix C and will be overwritten.
    If it is structured, for example, if it is positive definite, then the
    returned matrices B and C will have the same structure. Section 5 of the
    source explains what kind of structure can be used.

    Note:
        This corresponds to Framework 2 in Section 4 in the source.

    Source:
        "Approximation with Kronecker Products"
        by Van Loan, Pitsianis
        https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.42.1924&rep=rep1&type=pdf
    """
    if mats_A.ndim == 2:
        mats_A = np.expand_dims(mats_A, 0)
        init_C = np.expand_dims(init_C, 0)

    mats_B = np.zeros((mats_A.shape[0], rows_B, cols_B), dtype=mats_A.dtype)
    mats_C = init_C
    rows_C, cols_C = mats_C.shape[-2:]

    # Extract the blocks A_ij: Split the 3d array of shape (n, rows_B * rows_C, cols_B * cols_C) into a 5d array of
    # shape (n, rows_B, cols_B, rows_C, cols_C) where [:, i, j, :, :] corresponds to the blocks A_ij in equation (2).
    blocks_A = np.zeros((mats_A.shape[0], rows_B, cols_B, rows_C, cols_C), dtype=mats_A.dtype)
    for i, block_row in enumerate(np.split(mats_A, rows_B, axis=-2)):
        for j, block in enumerate(np.split(block_row, cols_B, axis=-1)):
            blocks_A[:, i, j, :, :] = block

    # Extract the blocks Ahat_ij: Split the 3d array of shape (n, rows_B * rows_C, cols_B * cols_C) into a 5d array of
    # shape (n, rows_C, cols_C, rows_B, cols_B) where [:, i, j, :, :] corresponds to the blocks Ahat_ij in equation (4).
    blocks_Ahat = np.zeros((mats_A.shape[0], rows_C, cols_C, rows_B, cols_B), dtype=mats_A.dtype)
    for i in range(rows_C):
        for j in range(cols_C):
            blocks_Ahat[:, i, j, :, :] = \
                mats_A[:, i: i + (rows_B - 1) * rows_C + 1: rows_C, j: j + (cols_B - 1) * cols_C + 1: cols_C]

    beta_or_gamma = np.zeros((mats_A.shape[0], 1, 1), dtype=mats_A.dtype)

    def project(blocks_ij, b_or_c, c_or_b_out):
        """
        For every block A_ij (or Ahat_ij), compute equation (8) (or (9)) in Theorem 4.1.
        """
        np.einsum('ijklm,ilm->ijk', blocks_ij, b_or_c, out=c_or_b_out)
        np.einsum('ijk,ijk->i', b_or_c, b_or_c, out=beta_or_gamma[:, 0, 0])
        c_or_b_out /= beta_or_gamma

    for _ in range(iterations):
        project(blocks_A, mats_C, mats_B)
        project(blocks_Ahat, mats_B, mats_C)

    return mats_B, mats_C


def kron_approx_svd(mats_A: np.ndarray, rows_B: int, cols_B: int, rows_C: int, cols_C: int):
    r"""
    Approximate a matrix in terms of a Kronecker product of two matrices.

    Note:
        Given a matrix A, find matrices B and C of shapes (rows_B, cols_B) and
        (rows_C, cols_C) such that \| A - B \otimes C \|_F is minimized.
        If A is structured, e.g., symmetric or positive definite, the function
        kron_approx_sep_ls can be used.

    Source:
        "Approximation with Kronecker Products"
        by Van Loan, Pitsianis
        https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.42.1924&rep=rep1&type=pdf
    """
    if mats_A.ndim == 2:
        mats_A = np.expand_dims(mats_A, 0)
    n, rows_A, cols_A = mats_A.shape
    if rows_B * rows_C != rows_A:
        raise ValueError(f'rows_B*rows_C = {rows_A} is required, but rows_B*rows_C = {rows_B*rows_C}')
    if cols_B * cols_C != cols_A:
        raise ValueError(f'cols_B*cols_C = {cols_A} is required, but cols_B*cols_C = {cols_B*cols_C}')

    def block(A, i, j):
        """
        Extract block A_ij from the matrix A. Every block A_ij has shape
        (rows_C, cols_C) and there are rows_B * cols_B such blocks.
        """
        return A[i * rows_C: (i + 1) * rows_C, j * cols_C: (j + 1) * cols_C]

    out_B = np.zeros((n, rows_B, cols_B), dtype=mats_A.dtype)
    out_C = np.zeros((n, rows_C, cols_C), dtype=mats_A.dtype)
    for ni in range(n):
        # the following implements equation (5)
        rearranged_A = np.zeros((rows_B * cols_B, rows_C * cols_C), dtype=mats_A.dtype)
        for j in range(cols_B):
            # extract the matrix A_j
            rearranged_A[j * rows_B: (j + 1) * rows_B, :] = np.concatenate(
                [block(mats_A[ni, :, :], i, j).flatten('F')[np.newaxis, :] for i in range(rows_B)],
                axis=0
            )
        u, s, vh = np.linalg.svd(rearranged_A)
        out_B[ni, :, :] = u[:, 0].reshape(rows_B, cols_B, order='F') * s[0]
        out_C[ni, :, :] = vh[0, :].reshape(rows_C, cols_C, order='F')
    return out_B, out_C


def kron_real(mats1: np.ndarray, mats2: np.ndarray):
    """
    Assuming mats1 and mats2 are real representations of complex covariance
    matrices, compute the real representation of the Kronecker product of the
    complex covariance matrices.
    """
    if mats1.ndim != mats2.ndim:
        raise ValueError(
            'The two arrays need to have the same number of dimensions, '
            f'but we have mats1.ndim = {mats1.ndim} and mats2.ndim = {mats2.ndim}.'
        )
    if mats1.ndim == 2:
        mats1 = np.expand_dims(mats1, 0)
        mats2 = np.expand_dims(mats2, 0)

    n = mats1.shape[0]
    rows1, cols1 = mats1.shape[-2:]
    rows2, cols2 = mats2.shape[-2:]
    row_half1 = rows1 // 2
    column_half1 = cols1 // 2
    row_half2 = rows2 // 2
    column_half2 = cols2 // 2
    rows3 = 2 * row_half1 * row_half2
    cols3 = 2 * column_half1 * column_half2

    out_kron_prod = np.zeros((n, rows3, cols3))
    for i in range(n):
        A1 = mats1[i, :row_half1, :column_half1]
        B1 = mats1[i, :row_half1, column_half1:]
        C1 = mats1[i, row_half1:, :column_half1]
        D1 = mats1[i, row_half1:, column_half1:]

        A2 = mats2[i, :row_half2, :column_half2]
        B2 = mats2[i, :row_half2, column_half2:]
        C2 = mats2[i, row_half2:, :column_half2]
        D2 = mats2[i, row_half2:, column_half2:]

        A = np.kron(A1 + D1, A2 + D2)
        D = -np.kron(C1 - B1, C2 - B2)
        B = -np.kron(A1 + D1, C2 - B2)
        C = np.kron(C1 - B1, A2 + D2)

        A = 0.5 * (A + D)
        D = A
        B = 0.5 * (B - C)
        C = -B

        out_kron_prod[i, :, :] = np.concatenate(
            (np.concatenate((A, B), axis=1), np.concatenate((C, D), axis=1)),
            axis=0
        )
    return np.squeeze(out_kron_prod)


def mat2bsc(mat: np.ndarray):
    """
    Arrange the real and imaginary parts of a complex matrix mat in block-
    skew-circulant form.

    Source:
        See https://ieeexplore.ieee.org/document/7018089.
    """
    upper_half = np.concatenate((mat.real, -mat.imag), axis=-1)
    lower_half = np.concatenate((mat.imag, mat.real), axis=-1)
    return np.concatenate((upper_half, lower_half), axis=-2)


def real2real(mats):
    re = np.real(mats)
    im = np.imag(mats)
    rows = mats.shape[1]
    cols = mats.shape[2]
    out = np.zeros([mats.shape[0], 2*rows, 2*cols])
    out[:, :rows, :cols] = 0.5*re
    out[:, rows:, cols:] = 0.5*re
    return out


def imag2imag(mats):
    im = np.real(mats)
    rows = mats.shape[1]
    cols = mats.shape[2]
    out = np.zeros([mats.shape[0], 2*rows, 2*cols])
    #out[:, :rows, :cols] = 0.5*re
    for i in range(mats.shape[0]):
        out[i, :rows, cols:] = 0.5*im[i,:,:].T
        out[i, rows:, :cols] = 0.5*im[i,:,:]
    #out[:, rows:, cols:] = 0.5*re
    return out


def nmse(actual: np.ndarray, desired: np.ndarray):
    """
    Mean squared error between actual and desired divided by the total number
    of elements.
    """
    mse = 0
    for i in range(actual.shape[0]):
        mse += np.linalg.norm(actual - desired) ** 2 / np.linalg.norm(desired) ** 2
    return mse / actual.shape[0]
    #return np.sum(np.abs(actual - desired) ** 2) / desired.size


def real2cplx(vec: np.ndarray, axis=0):
    """
    Assume vec consists of concatenated real and imaginary parts. Return the
    corresponding complex vector. Split along axis=axis.
    """
    re, im = np.split(vec, 2, axis=axis)
    return re + 1j * im


def sec2hours(seconds: float):
    """"
    Convert a number of seconds to a string h:mm:ss.
    """
    # hours
    h = seconds // 3600
    # remaining seconds
    r = seconds % 3600
    return '{:.0f}:{:02.0f}:{:02.0f}'.format(h, r // 60, r % 60)

def check_random_state(seed):
    import numbers
    """Turn seed into a np.random.RandomState instance

    Parameters
    ----------
    seed : None, int or instance of RandomState
        If seed is None, return the RandomState singleton used by np.random.
        If seed is an int, return a new RandomState instance seeded with seed.
        If seed is already a RandomState instance, return it.
        Otherwise raise ValueError.
    """
    if seed is None or seed is np.random:
        return np.random.mtrand._rand
    if isinstance(seed, numbers.Integral):
        return np.random.RandomState(seed)
    if isinstance(seed, np.random.RandomState):
        return seed
    raise ValueError('%r cannot be used to seed a numpy.random.RandomState'
                     ' instance' % seed)

def print_dict(dict: dict, entries_per_row: int=1):
    """Print the keys and values of dictionary dict."""
    if entries_per_row < 1:
        raise ValueError(f'The number of entries per row needs to be >= 1 but is {entries_per_row}')
    for c, (key, value) in enumerate(dict.items()):
        if c % entries_per_row == 0 and c > 0:
            print()
        else:
            c > 0 and print(' | ', end='')
        print('{}: {}'.format(key, value), end='')
    print()


def dft_matrix(n_antennas, n_grid):
    grid = np.linspace(-1, 1, n_grid + 1)[:n_grid]

    d = 1 / np.sqrt(n_antennas) * np.exp(1j * np.pi * np.outer(np.arange(n_antennas), grid.conj().T))
    return d

#def dist_fac(n_bits: int):
#    """"Compute distortion factor for an arbitrary number of bits."""
#    if n_bits == 1:
#        raise ValueError('The distortion factor for 1-bit is known in closed form.')
#    else:
#        return n_bits * 2**(-2*n_bits)

In [2]:
def scm_channel(
    angles,
    weights,
    n_coherence,
    n_antennas,
    sigma=2.0,
    rng=np.random.default_rng()
):
    (h, t) = chan_from_spectrum(n_coherence, n_antennas, angles, weights, sigma, rng=rng)
    return h, t


def spectrum(u, angles, weights, sigma=2.0):
    u = (u + np.pi) % (2 * np.pi) - np.pi
    theta = np.degrees(np.arcsin(u / np.pi))
    v = _laplace(theta, angles, weights, sigma) \
        + _laplace(180 - theta, angles, weights, sigma)

    return np.degrees(2 * np.pi * v / np.sqrt(np.pi ** 2 - u ** 2))


def _laplace(theta, angles, weights, sigma=2.0):
    # The variance \sigma^2 of a Laplace density is \sigma^2 = 2 * scale_parameter^2.
    # Hence, the standard deviation \sigma is \sigma = sqrt(2) * scale_parameter.
    # The scale_parameter determines the Laplace density.
    # For an angular spread (AS) given in terms of a standard deviation \sigma
    # the scale parameter thus needs to be computed as scale_parameter = \sigma / sqrt(2)
    scale_parameter = sigma / np.sqrt(2)
    x_shifted = np.outer(theta, np.ones(angles.size)) - angles
    x_shifted = (x_shifted + 180) % 360 - 180
    v = weights / (2 * scale_parameter) * np.exp(-np.absolute(x_shifted) / scale_parameter)
    return v.sum(axis=1)

# def spectrum(u, angles, weights, sigma=2.0):
#     pi = tf.constant(np.pi, dtype=tf.float32)
#     u = (u + pi) % (2 * pi) - pi
#     theta = (180.0 / pi) * tf.math.asin(u / pi)
#     v = _laplace(theta, angles, weights, sigma) + _laplace(180 - theta, angles, weights, sigma)

#     return (360.0 / pi) * (2 * pi * v / tf.sqrt(pi**2 - u**2))


# def _laplace(theta, angles, weights, sigma=2.0):
#     # The variance \sigma^2 of a Laplace density is \sigma^2 = 2 * scale_parameter^2.
#     # Hence, the standard deviation \sigma is \sigma = sqrt(2) * scale_parameter.
#     # The scale_parameter determines the Laplace density.
#     # For an angular spread (AS) given in terms of a standard deviation \sigma
#     # the scale parameter thus needs to be computed as scale_parameter = \sigma / sqrt(2)
#     scale_parameter = sigma / tf.sqrt(tf.constant(2.0, dtype=tf.float32))
    
#     theta = tf.matmul(tf.reshape(theta, (theta.shape[0], 1)), tf.reshape(tf.ones_like(angles), (1, angles.shape[0])))

 
#     # Calculate x_shifted using tf.matmul
#     x_shifted = theta - tf.reshape(angles, (1, angles.shape[0]))

#     x_shifted = theta - angles
#     x_shifted = (x_shifted + 180) % 360 - 180
#     print('x_shifted: ', x_shifted.shape)
#     print('scale_parameter: ', scale_parameter.shape)
#     print('weights: ', weights.shape)
#     print('x_shifted: ', tf.exp(-tf.abs(x_shifted) / scale_parameter).shape)
#     v = (weights / (2 * scale_parameter)) * tf.exp(-tf.abs(x_shifted) / scale_parameter)
#     return tf.reduce_sum(v, axis=1)






def chan_from_spectrum(
    n_coherence,
    n_antennas,
    angles,
    weights,
    sigma=2.0,
    rng=np.random.default_rng()
):

    o_f = 100  # oversampling factor (ideally, would use continuous freq. spectrum...)
    n_freq_samples = o_f * n_antennas

    # Sample the spectrum which is defined in equation (78) with epsilon, try
    # to avoid sampling at -pi and pi, thus avoiding dividing by zero.
    epsilon = 1 / 3
    lattice = np.arange(epsilon, n_freq_samples+epsilon) / n_freq_samples * 2 * np.pi - np.pi #sampled between -pi,+pi
    fs = spectrum(lattice, angles, weights, sigma)
    fs = np.reshape(fs, [len(fs), 1])

    # Avoid instabilities due to almost infinite energy at some frequencies
    # (this should only happen at "endfire" of a uniform linear array where --
    # because of the arcsin-transform -- the angular psd grows to infinity).
    almost_inf_threshold = np.max([1, n_freq_samples])  # use n_freq_samples as threshold value...
    almost_inf_freqs = np.absolute(fs) > almost_inf_threshold
    # this should not/only rarely be entered due to the epsilon above; one might even want to increase the threshold
    # to, e.g., 30 * almost_inf_threshold

    # if any(np.absolute(fs) > 20 * almost_inf_threshold):
    #     print("almost inf: ", fs[almost_inf_freqs])

    fs[almost_inf_freqs] = almost_inf_threshold  # * np.exp(1j * np.angle(fs[almost_inf_freqs])) # only real values

    if np.sum(fs) > 0:
        fs = fs / np.sum(fs) * n_freq_samples  # normalize energy

    x = crandn(n_freq_samples, n_coherence, rng=rng)

    h = np.fft.ifft(np.sqrt(fs)*x, axis=0) * np.sqrt(n_freq_samples)
    h = h[0:n_antennas, :]

    # t is the first row of the covariance matrix of h (which is Toeplitz and Hermitian)
    t = np.fft.fft(fs, axis=0) / n_freq_samples
    t = t[0:n_antennas]
    t = np.reshape(t, n_antennas)

    return h.T, t


# def chan_from_spectrum(
#     n_coherence,
#     n_antennas,
#     angles,
#     weights,
#     sigma=2.0,
#     rng=None
# ):
#     # Set random seed for TensorFlow

#     if rng is None:
#         tf.random.set_seed(1235428719812346)
#         rng = tf.random.Generator.from_seed()
        

#     o_f = 100  # oversampling factor (ideally, would use continuous freq. spectrum...)
#     n_freq_samples = o_f * n_antennas

#     # Sample the spectrum using TensorFlow
#     epsilon = 1 / 3
#     lattice = tf.range(epsilon, n_freq_samples + epsilon, dtype=tf.float32) / n_freq_samples * 2 * np.pi - np.pi
#     fs = spectrum(lattice, angles, weights, sigma)  # Assuming you have a spectrum function
    
#     # Reshape fs
#     fs = tf.reshape(fs, [len(fs), 1])

#     # Avoid instabilities due to almost infinite energy at some frequencies
#     almost_inf_threshold = tf.maximum(1.0, tf.cast(n_freq_samples, dtype=tf.float32))
#     almost_inf_freqs = tf.abs(fs) > almost_inf_threshold

#     # Set values for almost infinite frequencies
#     fs = tf.where(almost_inf_freqs, almost_inf_threshold, fs)

#     # Normalize energy
#     fs = fs / tf.reduce_sum(fs) * n_freq_samples

#     # Generate complex random numbers in TensorFlow
#     x = crandn(n_freq_samples, n_coherence, rng=rng)

#     print('dtype of tf.cast(tf.math.sqrt(n_freq_sampöles), dtype=tf.complex64): ', tf.math.sqrt(tf.cast(n_freq_samples, dtype=tf.complex64)).dtype)
#     print('dtype of x: ', x.dtype)
#     print('dtype of multiplication: ', (tf.cast(tf.math.sqrt(fs), dtype=tf.complex64) * x).dtype)

#     # Compute channel response in TensorFlow
#     h = tf.signal.ifft(tf.cast(tf.math.sqrt(fs), dtype=tf.complex64) * x) * tf.math.sqrt(tf.cast(n_freq_samples, dtype=tf.complex64))
#     h = h[0:n_antennas, :]

#     # Compute t as the first row of the covariance matrix of h
#     t = tf.signal.fft(tf.cast(fs, dtype=tf.complex64)) / n_freq_samples
#     t = t[0:n_antennas]
#     t = tf.reshape(t, [n_antennas])

#     return tf.transpose(h), t

In [3]:
class SCMMulti:
    """Class to build a multi path channel model.

    This class defines a multi path channel model.

    Public Methods:

    Instance Variables:

    """

    def __init__(self, path_sigma=2.0, n_path=3):
        """Initialize multi path channel model.

        First, initialise all variables belonging to the multi path channel model.
        """
        self.path_sigma = path_sigma
        self.n_path = n_path

    def generate_channel(
        self,
        n_batches,
        n_coherence,
        n_antennas,
        rng=np.random.default_rng()
    ):
        """Generate multi path model parameters.

        Returns:
            A tuple (h, t) consisting of channels h with
                h.shape = (n_batches, n_coherence, n_antennas)
            and the first rows t of the covariance matrices with
                t.shape = (n_batches, n_antennas)
        """
        h = np.zeros([n_batches, n_coherence, n_antennas], dtype=np.complex64)
        t = np.zeros([n_batches, n_antennas], dtype=np.complex64)

        for i in range(n_batches):
            gains = rng.random(self.n_path)
            gains = gains / np.sum(gains, axis=0)
            angles = (rng.random(self.n_path) - 0.5) * 180

            h[i, :, :], t[i, :] = scm_channel(angles, gains, n_coherence, n_antennas, self.path_sigma, rng=rng)

        return h, t

    # def generate_channel(
    #     self,
    #     n_batches,
    #     n_coherence,
    #     n_antennas,
    #     rng=None
    # ):

    #     if rng is None:
    #         tf.random.set_seed(1235428719812346)
    #         rng = tf.random.Generator.from_seed()
        
    #     # n_batches is a symbolic tensor and i want to convert it to a tf.Tensor with the value of batch_size that i define in the main function
    #     n_batches = tf.convert_to_tensor(n_batches, dtype=tf.int32)

    #     print('n_batches: ', n_batches)

    #     h = tf.zeros([n_batches, n_coherence, n_antennas], dtype=tf.complex64)
    #     t = tf.zeros([n_batches, n_antennas], dtype=tf.complex64)

    #     for i in range(n_batches):
    #         gains = rng.uniform(shape=(self.n_path,))
    #         gains = gains / tf.reduce_sum(gains)
    #         angles = (rng.uniform(shape=(self.n_path,)) - 0.5) * 180.0

    #         # Assuming you have a scm_channel function, replace with your actual implementation
    #         h_i, t_i = scm_channel(angles, gains, n_coherence, n_antennas, self.path_sigma, rng=rng)

    #         print('h_i: ', h_i.shape)
    #         print('h: ', h.shape)
    #         print('t_i: ', t_i.shape)

    #         h = tf.tensor_scatter_nd_add(h, [[i]], h_i)
    #         t = tf.tensor_scatter_nd_add(t, [[i]], t_i)

    #     return h, t

    def get_config(self):
        config = {
            'path_sigma': self.path_sigma,
            'n_path': self.n_path
        }
        return config


In [4]:
from scipy.linalg import toeplitz
def channel_generation(batch_size, n_coherences, n_antennas):
    """
    SIMO version.
    """
    path_sigma = 2.0
    n_path = 256
    channel = SCMMulti(path_sigma=path_sigma, n_path=n_path)

    # generate channel samples with a certain batch size
    rng = np.random.default_rng(1235428719812346)

    h, t = channel.generate_channel(batch_size, n_coherences, n_antennas, rng)
        
    # Initialize an empty array to store the covariance matrices
    C = np.empty((batch_size, n_antennas, n_antennas), dtype=np.complex64)
    
    for i in range(batch_size):
        # Calculate the covariance matrix for each sample
        C[i, :, :] = toeplitz(t[i, :])
    
    # print('Generated ' + str(batch_size) + ' SIMO channel samples of size ' + str(n_antennas) + 'x1.')
    
    return h, C


# def channel_generation(batch_size, n_coherences, n_antennas, seed=1235428719812346):
#     path_sigma = 2.0
#     n_path = 256
#     # Set random seed for TensorFlow
#     tf.random.set_seed(seed)

#     # Create SCMMulti instance
#     channel = SCMMulti(path_sigma=path_sigma, n_path=n_path)

#     # Generate channel samples with a certain batch size
#     rng = tf.random.Generator.from_seed(seed)
#     h, t = channel.generate_channel(batch_size, n_coherences, n_antennas, rng=rng)

#     # Initialize an empty array to store the covariance matrices
#     C = tf.zeros((batch_size, n_antennas, n_antennas), dtype=tf.complex64)

#     for i in range(batch_size):
#         # Calculate the covariance matrix for each sample
#         t_i = t[i, :]
#         C_i = tf.linalg.toeplitz(t_i)
#         C = tf.tensor_scatter_nd_add(C, [[i]], tf.expand_dims(C_i, axis=0))

#     print('Generated ' + str(batch_size) + ' SIMO channel samples of size ' + str(n_antennas) + 'x1.')

#     return h, C


In [5]:
import tensorflow as tf

class cond_normal_channel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        
    def __call__(self, x, no, batch_size, n_coherence, n_antennas, h=None, C=None):
        if h is None or C is None:
            h, C = channel_generation(batch_size, n_coherence, n_antennas)
                        
        n = tf.TensorArray(tf.complex64, size=batch_size)

        for i in range(batch_size):
            noise_real_i = tf.random.normal(h[i].shape, dtype=tf.float32)
            noise_imag_i = tf.random.normal(h[i].shape, dtype=tf.float32)
            noise_i = tf.complex(noise_real_i, noise_imag_i)
            noise_i = noise_i * tf.cast(tf.sqrt(no / 2.0), dtype=tf.complex64)
            n = n.write(i, noise_i)
        n = n.stack()
        
        x = tf.reshape(x, [-1, 1, 1])
                        
        y = h * x + n
        
        #print('first 10 elements of y: ', y[0,0,:10])
                
        return y, h, C

2023-11-07 09:29:50.707304: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-07 09:29:50.737561: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-07 09:29:51.148419: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
class equalizer(tf.keras.Model):
    def __init__(self):
        super().__init__()

    def __call__(self, h_hat, y, no):
               
        norm_h_hat_squared = tf.reduce_sum(tf.square(tf.abs(h_hat)), axis=-1)
                        
        no_new = tf.math.divide_no_nan(
            no,
            norm_h_hat_squared
        )
                        
        inner_product_h_y = tf.reduce_sum(tf.matmul(y, tf.linalg.adjoint(h_hat)), axis=-1)
                                                
        x_hat = tf.math.divide_no_nan(
            inner_product_h_y,
            tf.cast(norm_h_hat_squared, dtype=tf.complex64)
        )
                        
        return x_hat, no_new


In [7]:
class genie_mmse_estimator(tf.keras.Model):
    def __init__(self):
        super().__init__()

    def __call__(self, y, no, C, pilot):
        
        #print('no: ', no)
                                
        # noise_var = no^2 * I. Be careful of the data types!
        noise_var = tf.cast(no * tf.eye(C.shape[-1], batch_shape=[C.shape[0]]), dtype=tf.complex64)
                                        
        # scaled_C = |p|^2 * C
        scaled_C = tf.cast(tf.math.abs(pilot) ** 2, dtype=tf.complex64) * tf.cast(C, dtype=tf.complex64)
        
        # print('check if scaled_C is hermitian', tf.reduce_all(tf.equal(scaled_C, tf.math.conj(tf.transpose(scaled_C, perm=[0, 2, 1])))))
        
        eigenvalues, eigenvectors = tf.linalg.eigh(scaled_C)
                        
        # Compute the inverse of (Lambda * noise_var)^-1
        inverse_lambda_noise_var = tf.linalg.inv(tf.linalg.diag(eigenvalues) + noise_var)

        # Compute the inverse of the sum
        # inverse = tf.matmul(tf.matmul(eigenvectors, inverse_lambda_noise_var), tf.transpose(eigenvectors, conjugate=True, perm=[0, 2, 1]))
        
        # print('shape of einsum of inverse_lambda_noise_var: ', tf.einsum('ijk,ilk->ijl', inverse_lambda_noise_var, tf.transpose(eigenvectors, conjugate=True, perm=[0, 1, 2])).shape)
        
        # inverse = tf.einsum('ijk,ikl->ijl', eigenvectors, tf.einsum('ijk,ilk->ijl', inverse_lambda_noise_var, tf.transpose(eigenvectors, conjugate=True, perm=[0, 1, 2])))

        inverse = tf.linalg.inv(scaled_C + noise_var)
        
        # print('where are inverse and inverse_2 the same: ', tf.reduce_sum(tf.where(tf.abs(inverse - inverse_2) > 1e-2, tf.ones_like(inverse), tf.zeros_like(inverse))))
        
        # print('eigenvalues: ', eigenvalues[0, :])
        # print('maximum value of eigenvalues: ', tf.math.reduce_max(tf.math.real(eigenvalues), axis=1)[0])
        # print('minimum value of eigenvalues: ', tf.math.reduce_min(tf.math.real(eigenvalues), axis=1)[0])          
                
        # scaled_C_2 = conj(p) * C
        scaled_C_2 = tf.math.conj(pilot) * tf.cast(C, dtype=tf.complex64)                        

        # matrix = scaled_C_2 * inverse
        matrix = tf.matmul(scaled_C_2, inverse)



                                                                                                                            
        # h_hat_mmse = (scaled_C_2 * inverse) * y. Be careful of the data types!
        h_hat_mmse = tf.matmul(matrix, tf.transpose(tf.cast(y, dtype=tf.complex64), perm=[0, 2, 1]))   
        h_hat_mmse_2 = tf.einsum('ijk,ilk->ijl', matrix, y) 

        # print('shape of y: ', y.shape)
        # print('shape of h_hat_mmse: ', h_hat_mmse.shape)

        # print('first 10 elements of y: ', tf.transpose(tf.cast(y, dtype=tf.complex64), perm=[0, 2, 1])[0, :10, 0])
        # print('first 10 elements of h_hat_mmse: ', h_hat_mmse[0, :10, 0])
        # print('first 10 dimensions of matrix: ', matrix[0, :10, :10])

        #compare h_hat_mmse and y
        # print('where are h_hat_mmse and y the same', tf.reduce_sum(tf.where(tf.abs(tf.transpose(h_hat_mmse, perm=[0, 2, 1]) - y) > 1, tf.ones_like(h_hat_mmse), tf.zeros_like(h_hat_mmse))))
        
        
        # print('where are h_hat_mmse and h_hat_mmse_2 the same', tf.reduce_sum(tf.where(tf.abs(h_hat_mmse - h_hat_mmse_2) > 1e-2, tf.ones_like(h_hat_mmse), tf.zeros_like(h_hat_mmse))))
                                                
        return tf.transpose(h_hat_mmse, perm=[0, 2, 1])

In [8]:
import tensorflow as tf

class ls_estimator(tf.keras.Model):
    def __init__(self):
        super().__init__()

    def __call__(self, y, x):
        h_hat_ls = tf.math.divide_no_nan(y, x)
                
        return h_hat_ls

In [9]:
try:
    import sionna as sn
except AttributeError:
    import sionna as sn


class end2endModel(tf.keras.Model):

    def __init__(self, num_bits_per_symbol, block_length, n_coherence, n_antennas, genie_estimator):
        super().__init__()
        
        self.genie_estimator = genie_estimator
        
        self.n_coherence = n_coherence
        self.n_antennas = n_antennas
        self.num_bits_per_symbol = num_bits_per_symbol
        self.block_length = block_length
        self.constellation = sn.mapping.Constellation("qam", self.num_bits_per_symbol)
        self.mapper = sn.mapping.Mapper(constellation=self.constellation)
        self.demapper = sn.mapping.Demapper("app", constellation=self.constellation)
        self.binary_source = sn.utils.BinarySource()
        self.channel = cond_normal_channel()
        
        self.ls_estimator = ls_estimator()
        self.mmse_estimator = genie_mmse_estimator()
        
        self.equalizer = equalizer()
        
        self.demapper = sn.mapping.Demapper("app", constellation=self.constellation, num_bits_per_symbol=self.num_bits_per_symbol)

    def __call__(self, batch_size, ebno_db):
        
        #pilot phase
        
        no = sn.utils.ebnodb2no(ebno_db,
                                num_bits_per_symbol=self.num_bits_per_symbol,
                                coderate=1.0)
        
        #print('value of no: ', no)

        batch_size = tf.constant(256, dtype=tf.int32)

        batch_size = tf.cast(batch_size, dtype=tf.int32)

        bits = self.binary_source([batch_size, self.block_length]) # Blocklength set to 1024 bits
        
        x = self.mapper(bits)
        
        # print('shape of x: ', x.shape)
                                
        pilot = tf.ones((batch_size, 1, 1), dtype=tf.complex64)
                        
        y_p, h, C = self.channel(pilot, no, batch_size, self.n_coherence, self.n_antennas)
              
        h_hat_ls = self.ls_estimator(y_p, pilot)

        h_hat_mmse = self.mmse_estimator(y_p, no, C, pilot)
        
        
                
        # print('difference between h_hat_ls and h: ', tf.reduce_sum(tf.abs(h_hat_ls - h)))
        # print('difference between h_hat_mmse and h: ', tf.reduce_sum(tf.abs(h_hat_mmse - tf.cast(h, dtype=tf.complex64))))
                                
        #uplink phase
                
        #x_data = all x except x[0][0] (x has shape (batch_size, block_length / num_bits_per_symbol))
        x_data = x[:, 1:]
        
        # print('shape of x_data: ', x_data.shape)
                                        
        y = []
        x_hat_ls = []
        x_hat_mmse = []
        no_ls_new = []
        no_mmse_new = []
        llr_ls = tf.TensorArray(dtype=tf.float32, size=tf.cast(self.block_length / self.num_bits_per_symbol - 1, dtype=tf.int32))
        llr_mmse = tf.TensorArray(dtype=tf.float32, size=tf.cast(self.block_length / self.num_bits_per_symbol - 1, dtype=tf.int32))
                
        for i in range(tf.shape(x_data)[1]):
            #y = h * x + n for all x except first one
            # print('x_data[0][i].shape: ', x_data[0][i].shape)
            y_i = self.channel(x_data[:, i], no, batch_size, self.n_coherence, self.n_antennas, h, C)[0]
            # print('y_i.shape: ', y_i.shape)
            y.append(y_i)
        
            x_hat_ls_i, no_ls_new_i = self.equalizer(h_hat_ls, y_i, no)
            # print('x_hat_ls_i.shape: ', x_hat_ls_i.shape)
            # print('no_ls_new_i.shape: ', no_ls_new_i.shape)
            x_hat_ls.append(x_hat_ls_i)
            no_ls_new.append(no_ls_new_i)
            
            # print('difference between x_hat_ls_i and x_data[:, i]: ', tf.reduce_sum(tf.abs(x_hat_ls_i - tf.reshape(x_data[:, i], [-1, 1]))))

            x_hat_mmse_i, no_mmse_new_i = self.equalizer(h_hat_mmse, y_i, no)
            x_hat_mmse.append(x_hat_mmse_i)
            no_mmse_new.append(no_mmse_new_i)
                        
            # print('difference between x_hat_mmse_i and x_data[:, i]: ', tf.reduce_sum(tf.abs(x_hat_mmse_i - tf.reshape(x_data[:, i], [-1, 1]))))
                        
            #print('value of x_hat_mmse_i: ', x_hat_mmse_i)
            #print('value of no_mmse_new_i: ', no_mmse_new_i[0])
            
            llr_ls_i = self.demapper([x_hat_ls_i, no_ls_new_i])
            #llr_ls = tf.concat([llr_ls, tf.reshape(llr_ls_i, (batch_size, 2, 1))], axis=2)
            llr_ls = llr_ls.write(i, llr_ls_i)
            
            llr_mmse_i = self.demapper([x_hat_mmse_i, no_mmse_new_i])
            # print('llr_mmse_i.shape: ', llr_mmse_i.shape)
            llr_mmse = llr_mmse.write(i, llr_mmse_i)
            #llr_mmse = tf.concat([llr_mmse, tf.reshape(llr_mmse_i, (batch_size, 2, 1))], axis=2)
        
         
        bits = bits[:, self.num_bits_per_symbol:]
        
        llr_ls = llr_ls.stack()
        #print('llr_ls first two elements: ', llr_ls[0][0][0], llr_ls[0][0][1])
        # print('shape of bits: ', bits.shape)
        llr_ls = tf.transpose(llr_ls, perm=[1, 0, 2])
        #print('llrs_ls.shape after transpose: ', llr_ls.shape)
        llr_mmse = llr_mmse.stack()
        # print('llr_mmse.shape: ', llr_mmse.shape)
        llr_mmse = tf.transpose(llr_mmse, perm=[1, 0, 2])
        #print('llrs_mmse.shape after transpose: ', llr_mmse.shape)
        
        
        llr_ls = tf.split(llr_ls, num_or_size_splits=2, axis=2)
        llr_mmse = tf.split(llr_mmse, num_or_size_splits=2, axis=2)
            
        
        llr_ls = tf.reshape(tf.stack(llr_ls, axis=2), bits.shape)
        llr_mmse = tf.reshape(tf.stack(llr_mmse, axis=2), bits.shape)
        
        # bits_hat = tf.where(llr_ls > 0.0, tf.ones_like(bits), tf.zeros_like(bits))
        
        # print("bits_hat before reshaping:\n", bits_hat[:3, :6])
        # #print("llr_mmse before reshaping:\n", llr_mmse[:3, :3, :])
        # print("bits:\n", bits[:3, :6])  # 6 columns to account for interleaving
        #print('shape of llr_ls after reshape: ', llr_ls.shape)
        #print('llr_ls first two elements after reshape: ', llr_ls[0][0], llr_ls[0][1])
        # print('shape of llr_mmse after reshape: ', llr_mmse.shape)
        
        # bits_hat = tf.where(llr_ls > 0.0, tf.ones_like(bits), tf.zeros_like(bits))
        
        # print('first 30 bits in bits_hat: ', bits_hat[1][:30])
        # print('first 30 bits in bits: ', bits[1][:30])
        
        # mismatched_indices = tf.where(tf.not_equal(bits, bits_hat))
        # print(mismatched_indices[100: 150])

                        
        
        if self.genie_estimator:
            return bits, llr_mmse
        
        return bits, llr_ls

2023-11-07 09:29:51.828394: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: UNKNOWN ERROR (34)


In [ ]:
num_bits_per_symbol = 2
block_length = 128
ebno_db_min = -10.0 # Minimum value of Eb/N0 [dB] for simulations
ebno_db_max = 10.0 # Maximum value of Eb/N0 [dB] for simulations
batch_size = 128 # How many examples are processed by Sionna in parallel
n_coherence = 1
n_antennas = 32

uncoded_mmse_model = end2endModel(num_bits_per_symbol=num_bits_per_symbol, block_length=block_length, n_coherence=n_coherence, n_antennas=n_antennas, genie_estimator=True)
    
ber_plots = sn.utils.PlotBER("SC over 3GPP channel")
ber_plots.simulate(
    uncoded_mmse_model,
    ebno_dbs=np.linspace(ebno_db_min, ebno_db_max, 50),
    batch_size=batch_size,
    num_target_block_errors=100, # simulate until 100 block errors occured
    legend="Genie MMSE",
    soft_estimates=True,
    max_mc_iter=10, # run 100 Monte-Carlo simulations (each with batch_size samples)
    show_fig=False   
)

uncoded_ls_model = end2endModel(num_bits_per_symbol=num_bits_per_symbol, block_length=block_length, n_coherence=n_coherence, n_antennas=n_antennas, genie_estimator=False)

ber_plots.simulate(
    uncoded_ls_model,
    ebno_dbs=np.linspace(ebno_db_min, ebno_db_max, 50),
    batch_size=batch_size,
    num_target_block_errors=100, # simulate until 100 block errors occured
    legend="LSE",
    soft_estimates=True,
    max_mc_iter=10, # run 100 Monte-Carlo simulations (each with batch_size samples)
    show_fig=False
)

ber_plots()

EbNo [dB] |        BER |       BLER |  bit errors |    num bits | block errors |  num blocks | runtime [s] |    status
---------------------------------------------------------------------------------------------------------------------------------------
    -10.0 | 1.6133e-01 | 1.0000e+00 |        5204 |       32256 |          256 |         256 |        11.3 |reached target block errors
   -9.592 | 1.5002e-01 | 9.8828e-01 |        4839 |       32256 |          253 |         256 |        10.5 |reached target block errors
   -9.184 | 1.2602e-01 | 9.8438e-01 |        4065 |       32256 |          252 |         256 |        10.2 |reached target block errors
   -8.776 | 1.0420e-01 | 9.7266e-01 |        3361 |       32256 |          249 |         256 |        10.2 |reached target block errors
   -8.367 | 9.6416e-02 | 9.6875e-01 |        3110 |       32256 |          248 |         256 |        10.2 |reached target block errors
   -7.959 | 8.1566e-02 | 9.5703e-01 |        2631 |       32256 |